In [17]:
import os
os.chdir("x:\DL\Projects\MLOPs")
%pwd

'x:\\DL\\Projects\\MLOPs'

In [18]:
print(type(0.01))

<class 'float'>


In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file_dir: Path
    req_files: list
    
@dataclass(frozen=True)
class TrainLogConfig:
    model: str
    save_path: Path
    mlflow_uri: str
    experiment_name: str
    model_name: str

@dataclass(frozen=True)
class Params:
    optimizer: str
    lr0: float
    save_period: int
    batch: int
    epochs: int
    resume: bool
    seed: int
    imgsz: int 

In [27]:
from scripts.MLOPs.constants import *
from scripts.MLOPs.utils.common import read_yaml, create_directories
#from scripts.MLOPs.entity.config_entity import *

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_dataingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data= config.local_data,
            unzip_dir= config.unzip_dir
            )
        return data_ingestion_config
    
    def get_datavalidation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.data_val_dir])
        data_validation_config = DataValidationConfig(
            current_dset= config.current_dset,
            root_dir=config.data_val_dir,
            status_file_dir= config.data_val_status,
            req_files= config.data_val_req
            )
        return data_validation_config
    
    def get_train_log_config(self)-> TrainLogConfig:
        config = self.config.train_log_config
        trainlogconfig = TrainLogConfig(
            model= config.model,
            save_path= config.save_path,
            mlflow_uri= config.mlflow_uri,
            experiment_name= config.experiment_name,
            model_name= config.model_name
        )
        return trainlogconfig
    
    def get_params(self)-> Params:
        param = self.config.params
        params = Params(
            optimizer = param.optimizer,
            lr0 = param.lr0,
            save_period = param.save_period,
            batch = param.batch,
            epochs = param.epochs,
            resume = param.resume,
            seed = param.seed,
            imgsz = param.imgsz
        )
        return params
    
    

In [28]:
from ultralytics import YOLO, settings
import os,sys
import logging
import torch
from urllib.parse import urlparse
import mlflow
from scripts.MLOPs.exception import AppException

class ModelTrainer:
    def __init__(self, config: TrainLogConfig, val: DataValidationConfig, param: Params):
        self.config = config
        self.val = val
        self.param = param

    def validation_status(self):
        with open(self.val.status_file_dir, 'r') as file:
            status = file.read().strip()
        key, value = status.split(':')
        key = key.strip()
        value = value.strip().lower()

        if key != "validation_status":
            raise ValueError("unexpected key in status file")
        
        if value == 'true':
            return True
        elif value == 'false':
            return False
        else:
            raise ValueError("validation status is invalid")

    def train_model(self):

        dataset_dir = self.val.current_dset
        data_path = os.path.join( dataset_dir, "data.yaml")
        logging.info(f"Dataset location: {data_path}")
        if torch.cuda.is_available():
            device = torch.cuda.current_device()
            logging.info(f"Device is running on: {torch.cuda.get_device_name(device)}")
        else:
            logging.info(f"CUDA is not available")
            device = "cpu"
            logging.info(f"Device to run on: {device}")
            logging.info(data_path)

        model = self.config.model

        os.makedirs(self.config.save_path, exist_ok=True)
        save_path = os.path.join(self.config.save_path,"Trainedv8.pt")
        # Load a pretrained YOLOv8n model
        model = YOLO(model)
        # Train the model
        model.train(
            data=data_path,
            optimizer = self.param.optimizer,
            lr0 = self.param.lr0,
            save_period = self.param.save_period,
            batch = self.param.batch,
            epochs = self.param.epochs,
            resume = self.param.resume,
            seed = self.param.seed,
            imgsz = self.param.imgsz
            )
        model.save(save_path)
        return model
    
    def log_into_mlflow(self):

        settings.update({'mlflow': True})
        run_name = self.config.model_name
        experiment_name = self.config.experiment_name
        os.environ["MLFLOW_TRACKING_URI"] = self.config.mlflow_uri
        os.environ["MLFLOW_RUN"] = run_name
        
        print("MLFLOW_TRACKING_URI: ", os.environ.get("MLFLOW_TRACKING_URI"))
        
        # settings.update({'datasets_dir': "X:\DL\Projects\MLOPs"})
        mlflow.autolog(log_models=True)
        mlflow.set_experiment(experiment_name=experiment_name)
        
        with mlflow.start_run(run_name=run_name) as run: 
            #run_name is the name of the task.
            run_id = run.info.run_id 
            #run_id is the directory name that will be stored within the MLFLOW_TRACKING_URI path.
            print(run_id)
            self.train_model()
            save_path = os.path.join(self.config.save_path,"Trainedv8.pt")
            model = YOLO(save_path)
            model.export(format="onnx", dynamic= True)
            mlflow.onnx.log_model(model, "YOLOv8n")             ##logged model
        mlflow.end_run()
            
    
    
    # def run_pipeline(self):
    #     try:
    #         with open()
    #         self.train_model()
    #     except Exception as e:
    #         raise AppException(e, sys)

    def run_pipeline(self):
        if (self.validation_status() == True):
            try:
                self.log_into_mlflow()
            except Exception as e:
                raise AppException(e, sys)
        else:
            logging.INFO(f"Model training not run due to invalid dataset. \n please ingest a valid dataset")


In [29]:
try:
    obj2 = ConfigurationManager()
    model_config = obj2.get_train_log_config()
    valc = obj2.get_datavalidation_config()
    params = obj2.get_params()
    x = ModelTrainer(config=model_config, val=valc, param=params)
    x.run_pipeline()
except Exception as e:
    raise AppException(e, sys)

[2024-08-01 21:40:04,393: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-08-01 21:40:04,396: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-08-01 21:40:04,396: INFO: common: created directory at artifacts]
[2024-08-01 21:40:04,397: INFO: common: created directory at artifacts/data_validation]


AppException: Error occured in python script name [C:\Users\aravi\AppData\Local\Temp\ipykernel_11756\2631944331.py] line number [4] error message [__init__() got an unexpected keyword argument 'current_dset']

AttributeError: 'ModelTrainer' object has no attribute 'log_into_mlflow'

In [10]:
try:
    obj2 = ConfigurationManager()
    model_config = obj2.get_model_trainer()
    dirc = obj2.get_dataingestion_config()
    x = ModelTrainer(config=model_config, dir=dirc)
    x.train_model()
except Exception as e:
    raise AppException(e, sys)


[2024-07-29 00:59:56,601: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-07-29 00:59:56,604: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-07-29 00:59:56,604: INFO: common: created directory at artifacts]
[2024-07-29 00:59:56,605: INFO: common: created directory at artifacts/data_ingestion]
[2024-07-29 00:59:56,606: INFO: 1550852978: Dataset location: artifacts/data_ingestion/data\data.yaml]
[2024-07-29 00:59:56,607: INFO: 1550852978: CUDA is not available]
[2024-07-29 00:59:56,607: INFO: 1550852978: Device to run on: cpu]
[2024-07-29 00:59:56,607: INFO: 1550852978: artifacts/data_ingestion/data\data.yaml]
New https://pypi.org/project/ultralytics/8.2.68 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=artifacts/data_ingestion/data\data.yaml, epochs=1, time=None, patience=100, batch=

c:\Users\aravi\anaconda3\envs\MLOPs\lib\site-packages\ultralytics\engine\trainer.py:268: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\train\labels... 2326 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2326/2326 [00:06<00:00, 354.57it/s]


train: New cache created: X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\train\labels.cache


val: Scanning X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\valid\labels... 331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 331/331 [00:01<00:00, 272.30it/s]


val: New cache created: X:\DL\Projects\MLOPs\artifacts\data_ingestion\data\valid\labels.cache
Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005078125), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train10
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.326      2.858      1.536          5        640: 100%|██████████| 466/466 [12:00<00:00,  1.55s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:42<00:00,  1.25s/it]


                   all        331        497      0.499      0.537      0.502      0.245

1 epochs completed in 0.213 hours.
Optimizer stripped from runs\detect\train10\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train10\weights\best.pt, 6.2MB

Validating runs\detect\train10\weights\best.pt...
Ultralytics YOLOv8.2.67  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:34<00:00,  1.03s/it]


                   all        331        497      0.513      0.529      0.505      0.246
              Backpack         95        117      0.721      0.575      0.706      0.359
             Cellphone         55         76      0.395      0.474       0.36      0.214
                 Drill         79         81      0.394      0.469      0.408      0.162
     Fire extinguisher         85         95      0.468      0.574      0.507      0.211
              Survivor        117        128      0.589      0.555      0.542      0.286
Speed: 2.5ms preprocess, 84.9ms inference, 0.0ms loss, 8.2ms postprocess per image
Results saved to runs\detect\train10
